In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
sns.set()
from sklearn.linear_model import LinearRegression
from statsmodels.regression.linear_model import OLS
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [3]:
#train = pd.read_csv('./data_1/Fina_Train.csv')
#test = pd.read_csv('./data_1/Fina_Test.csv')
train = pd.read_csv('Train-19-10.csv')
test = pd.read_csv('Test-19-10.csv')

In [4]:
y = train['TARGET(PRICE_IN_LACS)']
x = train.drop(['TARGET(PRICE_IN_LACS)','BHK_OR_RK','ADDRESS','CITY','Lo+Li', 'Rare_House','READY_TO_MOVE', 'SQFT_P_ROOM', 'Lo+Li', 'Rare_House', 'City_Type'],axis=1)


In [5]:
train.columns

Index(['POSTED_BY', 'UNDER_CONSTRUCTION', 'RERA', 'BHK_NO.', 'BHK_OR_RK',
       'SQUARE_FT', 'READY_TO_MOVE', 'RESALE', 'ADDRESS', 'LONGITUDE',
       'LATITUDE', 'TARGET(PRICE_IN_LACS)', 'CITY', 'IN_CITY', 'CITY_2',
       'SQFT_P_ROOM', 'Lo+Li', 'Rare_House', 'City_Type'],
      dtype='object')

In [6]:
total_col = ['POSTED_BY', 'UNDER_CONSTRUCTION', 'RERA', 'BHK_NO.', 'BHK_OR_RK',
       'SQUARE_FT',  'RESALE', 'ADDRESS', 'LONGITUDE',
       'LATITUDE', 'CITY', 'CITY_2']

In [7]:
cat_col = ['POSTED_BY','CITY_2','IN_CITY']

In [8]:
x.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,SQUARE_FT,RESALE,LONGITUDE,LATITUDE,IN_CITY,CITY_2
0,Owner,0,0,2,7.170301,1,12.969910,4.351541,Ksfc Layout,Bangalore
1,Dealer,0,0,2,7.150701,1,12.274538,4.339179,Vishweshwara Nagar,Mysore
2,Owner,0,0,2,6.838576,1,12.778033,4.351982,Jigani,Bangalore
3,Owner,0,1,2,6.835100,1,28.642300,4.348269,Sector-1 Vaishali,Ghaziabad
4,Owner,0,0,3,7.130899,1,10.033280,4.334444,South Chittoor,Kochi


In [9]:
#train['City_Type'].value_counts()

In [10]:
bhk_or_rak = {'BHK':1,
             'RK':0}
posted_by = {'Dealer':0,'Owner':1,'Builder':2}
city_typ = {'Metro':0,'Town':1,'Village':2,'City':3,'Unknown':4}

In [11]:
for df in [x,test]:
    #df['BHK_OR_RK'] = df['BHK_OR_RK'].replace(bhk_or_rak)
    df['POSTED_BY'] = df['POSTED_BY'].replace(posted_by)
    #df['City_Type'] = df['City_Type'].replace(city_typ)

In [12]:
city_encode = LabelEncoder()
city_encode.fit(list(set(x['CITY_2']) | set(test['CITY_2'])))
in_city_encode = LabelEncoder()
in_city_encode.fit(list(set(x['IN_CITY']) | set(test['IN_CITY'])))
for df in [x,test]:
    df['CITY_2'] = city_encode.transform(df['CITY_2'] )
    df['IN_CITY'] = in_city_encode.transform(df['IN_CITY'].astype(str))

In [13]:
#final_x = x.drop(['LATITUDE','LONGITUDE'],axis=1)


In [14]:
'''plt.figure(figsize=(16,16))
sns.heatmap(final_x.merge(y,on=x.index).drop(['key_0'],axis=1).corr(),cmap='coolwarm',annot=True,square=True)'''

"plt.figure(figsize=(16,16))\nsns.heatmap(final_x.merge(y,on=x.index).drop(['key_0'],axis=1).corr(),cmap='coolwarm',annot=True,square=True)"

In [15]:
x.columns

Index(['POSTED_BY', 'UNDER_CONSTRUCTION', 'RERA', 'BHK_NO.', 'SQUARE_FT',
       'RESALE', 'LONGITUDE', 'LATITUDE', 'IN_CITY', 'CITY_2'],
      dtype='object')

In [17]:
#final_x = x.drop()
final_x = x.copy()
#final_x = x.drop([  'BHK_OR_RK','READY_TO_MOVE'],axis=1)


In [34]:
final_x['LONGITUDE'] = np.log(final_x['LONGITUDE'])

In [35]:
final_x.shape

(24736, 10)

## Split

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [37]:
'''scaler = StandardScaler().fit(final_x)
final_x = scaler.transform(final_x)'''

'scaler = StandardScaler().fit(final_x)\nfinal_x = scaler.transform(final_x)'

In [38]:
x_train,x_test,y_train,y_test = train_test_split(final_x,y,train_size=0.7,random_state=22)

x_test,x_valid,y_test,y_valid = train_test_split(x_test,y_test,train_size=0.5,random_state=2)
x_train.shape,x_test.shape,x_valid.shape

((17315, 10), (3710, 10), (3711, 10))

### XGBoost

In [57]:
from xgboost import XGBRegressor,plot_importance,plot_tree
import xgboost as xgb

In [40]:
'''reg = XGBRegressor(n_estimators=2000,objective='reg:squaredlogerror',verbosity=1,tree_method='exact',n_jobs=-1)
score = reg.fit(final_x,y)
pred = score.predict(final_x)
train_acc = 100-mean_squared_log_error(np.exp(y),np.exp(pred))*100
#test_acc = 100-mean_squared_log_error(np.exp(y_test),np.exp(score.predict(x_test)))*100
print('Train',train_acc)
#print('Test',test_acc)
'''

"reg = XGBRegressor(n_estimators=2000,objective='reg:squaredlogerror',verbosity=1,tree_method='exact',n_jobs=-1)\nscore = reg.fit(final_x,y)\npred = score.predict(final_x)\ntrain_acc = 100-mean_squared_log_error(np.exp(y),np.exp(pred))*100\n#test_acc = 100-mean_squared_log_error(np.exp(y_test),np.exp(score.predict(x_test)))*100\nprint('Train',train_acc)\n#print('Test',test_acc)\n"

In [41]:
x_train.shape

(17315, 10)

In [42]:
max_depth = [ 6,7,8,9,]
eta = [0.2,0.3,0.4]
early_stopping_rounds = 20

In [102]:
#reg = XGBRegressor(objective='reg:squaredlogerror' ,n_jobs=-1,n_estimators=500,verbosity=1,eta=0.4,gamma=0.001,max_depth= 6, min_child_weight= 1)
eval_set = [(x_train, y_train),(x_test, y_test),(x_valid,y_valid) ]
reg = XGBRegressor(objective='reg:squaredlogerror' ,n_jobs=-1,n_estimators=1000,eta=0.4,max_depth=6,gamma=0.001,min_child_weight= 0.001,subsample=0.5)

score = reg.fit(x_train,y_train,verbose=True,eval_metric=['rmsle'],eval_set=eval_set,early_stopping_rounds=25)
pred = score.predict(x_train)
train_acc = 100-mean_squared_log_error(np.exp(y_train),np.exp(pred))*100
test_acc = 100-mean_squared_log_error(np.exp(y_test),np.exp(score.predict(x_test)))*100
#train_acc = 100-mean_squared_log_error(y_train,pred)*100
#test_acc = 100-mean_squared_log_error(y_test,score.predict(x_test))*100

print('Train',train_acc)
print('Test',test_acc)


[0]	validation_0-rmsle:1.04385	validation_1-rmsle:1.04065	validation_2-rmsle:1.03979
Multiple eval metrics have been passed: 'validation_2-rmsle' will be used for early stopping.

Will train until validation_2-rmsle hasn't improved in 25 rounds.
[1]	validation_0-rmsle:0.86047	validation_1-rmsle:0.85740	validation_2-rmsle:0.85665
[2]	validation_0-rmsle:0.69422	validation_1-rmsle:0.69134	validation_2-rmsle:0.69073
[3]	validation_0-rmsle:0.54690	validation_1-rmsle:0.54440	validation_2-rmsle:0.54385
[4]	validation_0-rmsle:0.42092	validation_1-rmsle:0.41894	validation_2-rmsle:0.41841
[5]	validation_0-rmsle:0.31693	validation_1-rmsle:0.31551	validation_2-rmsle:0.31477
[6]	validation_0-rmsle:0.23530	validation_1-rmsle:0.23471	validation_2-rmsle:0.23361
[7]	validation_0-rmsle:0.17459	validation_1-rmsle:0.17475	validation_2-rmsle:0.17353
[8]	validation_0-rmsle:0.13231	validation_1-rmsle:0.13330	validation_2-rmsle:0.13129
[9]	validation_0-rmsle:0.10461	validation_1-rmsle:0.10648	validation_2-rms

[94]	validation_0-rmsle:0.04732	validation_1-rmsle:0.05944	validation_2-rmsle:0.05528
[95]	validation_0-rmsle:0.04725	validation_1-rmsle:0.05935	validation_2-rmsle:0.05521
[96]	validation_0-rmsle:0.04717	validation_1-rmsle:0.05937	validation_2-rmsle:0.05521
[97]	validation_0-rmsle:0.04703	validation_1-rmsle:0.05927	validation_2-rmsle:0.05508
[98]	validation_0-rmsle:0.04694	validation_1-rmsle:0.05927	validation_2-rmsle:0.05506
[99]	validation_0-rmsle:0.04684	validation_1-rmsle:0.05925	validation_2-rmsle:0.05502
[100]	validation_0-rmsle:0.04678	validation_1-rmsle:0.05928	validation_2-rmsle:0.05500
[101]	validation_0-rmsle:0.04670	validation_1-rmsle:0.05926	validation_2-rmsle:0.05496
[102]	validation_0-rmsle:0.04660	validation_1-rmsle:0.05929	validation_2-rmsle:0.05498
[103]	validation_0-rmsle:0.04652	validation_1-rmsle:0.05928	validation_2-rmsle:0.05496
[104]	validation_0-rmsle:0.04642	validation_1-rmsle:0.05920	validation_2-rmsle:0.05491
[105]	validation_0-rmsle:0.04634	validation_1-rms

[189]	validation_0-rmsle:0.04045	validation_1-rmsle:0.05745	validation_2-rmsle:0.05413
[190]	validation_0-rmsle:0.04034	validation_1-rmsle:0.05751	validation_2-rmsle:0.05412
[191]	validation_0-rmsle:0.04027	validation_1-rmsle:0.05748	validation_2-rmsle:0.05413
Stopping. Best iteration:
[166]	validation_0-rmsle:0.04175	validation_1-rmsle:0.05780	validation_2-rmsle:0.05404

Train 95.36335114107116
Test 91.57302509216171


In [69]:
reg.get_xgb_params()

{'objective': 'reg:squaredlogerror',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'monotone_constraints': None,
 'n_jobs': -1,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None,
 'eta': 0.4}

In [115]:
params = {
    'min_child_weight':[4,5],
    'gamma':[i/10.0 for i in range(3,6)],
    'subsample':[i/10.0 for i in range(6,11)],
    'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [3,4,5,6,7,8,9],
    'objective': ['reg:squaredlogerror'],
    'eval_metric':['rmsle']
    
}

In [116]:
from sklearn.model_selection import GridSearchCV

In [120]:
import sklearn

In [119]:
sklearn.metrics.SCORERS.keys()

NameError: name 'sklearn' is not defined

In [118]:
xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params,scoring='rmsle',cv=4,verbose=1)
grid.fit(x_train, y_train)


ValueError: 'rmsle' is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.

In [67]:
data_dmatrix = xgb.DMatrix(data=x_train,label=y_train)
xgb_param = reg.get_xgb_params()
#do cross validation
print ('Start cross validation')
cvresult = xgb.cv(xgb_param, data_dmatrix, num_boost_round=100, nfold=5, metrics=['rmsle'],early_stopping_rounds=20, seed=1301,verbose_eval=2)

Start cross validation
[0]	train-rmsle:1.04374+0.00061	test-rmsle:1.04377+0.00254
[2]	train-rmsle:0.69380+0.00050	test-rmsle:0.69382+0.00233
[4]	train-rmsle:0.41970+0.00036	test-rmsle:0.41972+0.00194
[6]	train-rmsle:0.23283+0.00016	test-rmsle:0.23310+0.00167
[8]	train-rmsle:0.12894+0.00014	test-rmsle:0.12966+0.00116
[10]	train-rmsle:0.08494+0.00017	test-rmsle:0.08640+0.00089
[12]	train-rmsle:0.07090+0.00033	test-rmsle:0.07302+0.00086
[14]	train-rmsle:0.06683+0.00043	test-rmsle:0.06939+0.00083
[16]	train-rmsle:0.06490+0.00062	test-rmsle:0.06763+0.00071
[18]	train-rmsle:0.06304+0.00033	test-rmsle:0.06606+0.00096
[20]	train-rmsle:0.06192+0.00040	test-rmsle:0.06532+0.00093
[22]	train-rmsle:0.06109+0.00050	test-rmsle:0.06476+0.00082
[24]	train-rmsle:0.06020+0.00039	test-rmsle:0.06412+0.00096
[26]	train-rmsle:0.05905+0.00035	test-rmsle:0.06325+0.00104
[28]	train-rmsle:0.05837+0.00040	test-rmsle:0.06282+0.00109
[30]	train-rmsle:0.05760+0.00054	test-rmsle:0.06229+0.00106
[32]	train-rmsle:0.056

In [68]:
cvresult

,train-rmsle-mean,train-rmsle-std,test-rmsle-mean,test-rmsle-std
0,1.043745,0.000605,1.043767,0.002541
1,0.860322,0.000554,0.860356,0.002477
2,0.693795,0.000498,0.693821,0.002326
3,0.546283,0.000431,0.546309,0.002162
4,0.419696,0.000361,0.419724,0.001942
...,...,...,...,...
95,0.047014,0.000488,0.056898,0.001019
96,0.046940,0.000490,0.056895,0.001015
97,0.046836,0.000467,0.056871,0.001044
98,0.046737,0.000492,0.056833,0.001043


In [ ]:
validation_0-rmsle:0.04115	validation_1-rmsle:0.05546	validation_2-rmsle:0.05218


validation_0-rmsle:0.02974	validation_1-rmsle:0.05053	validation_2-rmsle:0.04940

reg = XGBRegressor(objective='reg:squaredlogerror' ,n_jobs=-1,n_estimators=1000,eta=0.3,max_depth=9,gamma=0.00001)

score = reg.fit(x_train,y_train,verbose=True,eval_metric=['rmsle'],eval_set=eval_set,early_stopping_rounds=20)


results = reg.evals_result()
epochs = len(results['validation_0']['rmsle'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = plt.subplots(figsize=(16,8))
ax.plot(x_axis, results['validation_0']['rmsle'], label='Train')
ax.plot(x_axis, results['validation_1']['rmsle'], label='Test')
ax.plot(x_axis, results['validation_1']['rmsle'], label='Valid')

ax.legend()
plt.ylabel('rmsle ')
plt.title('XGBoost rmsle ')
plt.show()

In [ ]:
#fig,ax = plt.subplots(figsize=(12,12))
plot_importance(reg)

## Cross Validation

In [ ]:
from sklearn.model_selection import KFold,cross_validate

In [ ]:
scv = KFold(n_splits=5,shuffle=True,random_state=37)

In [ ]:
score = cross_validate(reg,final_x,y,scoring='neg_mean_squared_log_error',cv=scv,n_jobs=3,verbose=2)

In [ ]:
-1*score['test_score']

In [ ]:
plt.figure(figsize=(16,8))
sns.lineplot(np.exp(y_train),np.exp(pred),alpha=0.2)

## Prediction

In [ ]:
#test = pd.read_csv('./Test-19-10.csv')
test = test.drop(['ADDRESS','CITY'],axis=1)

In [ ]:
test.head()

In [ ]:
#encoded_cat = encoder.transform(test[cat_col])

In [ ]:
#temp =pd.DataFrame(data=encoded_cat.toarray(),columns=col_name)

In [ ]:
pd.options.display.max_rows = None

In [ ]:
'''import bisect
le_classes = l_encode.classes_.tolist()
bisect.insort_left(le_classes, 'unknown')
l_encode.classes_ = le_classes'''

In [ ]:
'''test[test['CITY_2'].map(lambda x: False if x  in city_encode.classes_ else True)]'''

In [ ]:
#test['CITY_2'] = test['CITY_2'].map(lambda x : 'unknown' if x not in l_encode.classes_ else x)

In [ ]:
#test.iloc[2274,-2] = 'Ravapar'

In [ ]:
#test['IN_CITY'].isna().sum()

In [ ]:
test.head()

In [ ]:
test[(test.CITY_2 == 190)]

In [ ]:
test['LONGITUDE'] = np.log(test['LONGITUDE'])
final_test = test.copy()

In [ ]:
#final_test = test.merge(temp,on=test.index).copy()

In [ ]:
#final_test = final_test.drop(['key_0','POSTED_BY','CITY_2','ZONE','UNDER_CONSTRUCTION','ZONE_0', 'ZONE_1', 'ZONE_2', 'ZONE_3',
     #  'ZONE_4', 'ZONE_5', 'ZONE_6', 'ZONE_7', 'ZONE_8','POSTED_BY_Builder','CITY_2_Agra'],axis=1)

In [ ]:
final_test.shape

In [ ]:
final_test.head()

In [ ]:
final_test['TARGET(PRICE_IN_LACS)'] = np.exp(score.predict(final_test))

In [ ]:
final_test['TARGET(PRICE_IN_LACS)']

In [ ]:
final_test['TARGET(PRICE_IN_LACS)'].to_csv('Submission 15.csv',index=False)

In [ ]:
!pip install graphviz

In [ ]:
import os
os.environ["PATH"] += os.pathsep + 'D:\\Downloads\\Chrome\\graphviz-2.44.1-win32\\Graphviz\\bin\\'

In [ ]:
from graphviz import Source

In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
plot_tree(reg,num_trees=5,ax=ax)